# MAT281

## Aplicaciones de la Matemática en la Ingeniería

### Tarea - Módulos 3 y 4

## Parte 2 - Gapminder con más de una fuente de datos [30 puntos]

La siguiente tarea debe realizarse de manera individual. No es aceptable **bajo ninguna circunstancia** copiar y pegar código de otros estudiantes. Pueden consultar referencias en internet (stackoverflow, por ejemplo), pero deben indicar sus fuentes cuando corresponda y evitar copiar y pegar texto y código.
Todo el código debe ser ejecutable y las fórmulas en markdown y latex.

Enviar los archivos correspondientes a las partes 1, 2, 3 y 4 comprimidos en un zip, por correo.

**Nombre**: *[COLOCAR NOMBRE ACA]*

**Rol**: *[COLOCAR ROL ACA]*


**Fecha de entrega**: Martes 18 Diciembre 2018.

Esta tarea usa el mismo ambiente previamente generado para la parte 1.

**Test de librerías**:

Usaremos las mismas librerías que en la parte 1. 

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

# Introducción

Uno de los principales gráficos mostrados es el siguiente:


![EjemploIPYWIDGET](images/fertility.png "Ejemplo de ipywidget")



Como puede apreciarse, el gráfico utiliza datos que no existen en el conjunto de datos `gapminder.csv`. Tendremos por tanto que ajustar un poco el ejemplo presentado en la parte 1 para poder reproducir este gráfico.

## Creando un dataset con datos de fertilidad

Consideremos los datos originales, tal y como los cargamos en la parte 1.

In [1]:
# Cargando los datos
import pandas as pd
import os

# Open the file
fpath_gapminder = os.path.join("data","gapminderData.csv")
df_gapminder_no_color = pd.read_csv(fpath_gapminder, sep=",")

# Rename the columns
df_gapminder_no_color = df_gapminder_no_color.rename(columns={"pop":"population", "lifeExp":"life_expectancy", "gdpPercap":"gdp_per_capita"})

# Create Continent-Color dataframe
data = [["Asia","red"],["Europe","orange"],["Africa", "b"],["Americas","y"],["Oceania","r"]]
df_color = pd.DataFrame(data, columns=["continent", "color"])
df_color

# Add Color
df_gapminder = df_gapminder_no_color.merge(df_color)

Carguemos los datos relativos a la fertilidad

In [2]:
fpath_fertility = os.path.join("data","indicator undata total_fertility.csv")
df_fertility_wide = pd.read_csv(fpath_fertility, sep=";")

Una rápida exploración nos muestra que tenemos los datos en formato "wide" y no en formato long como nos sería conveniente:

In [3]:
df_fertility_wide.head()

,fertility_rate,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,...,6.70,6.46,6.20,5.93,5.66,5.40,5.14,4.90,4.68,4.47
1,Albania,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,...,1.85,1.80,1.76,1.74,1.74,1.75,1.76,1.77,1.78,1.78
2,Algeria,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,...,2.58,2.66,2.73,2.78,2.82,2.83,2.82,2.80,2.76,2.71
3,Angola,6.93,6.93,6.93,6.93,6.93,6.93,6.93,6.94,6.94,...,6.60,6.52,6.43,6.33,6.22,6.10,5.98,5.86,5.75,5.65
4,Antigua and Barbuda,5.00,5.00,4.99,4.99,4.99,4.98,4.98,4.97,4.97,...,2.20,2.18,2.16,2.15,2.13,2.12,2.10,2.09,2.08,2.06


Para juntar ambos dataframes, necesitamos por tanto "despivotear" (melt) el dataframe.

In [4]:
years = df_fertility_wide.columns[1:]
df_fertility_long = pd.melt(df_fertility_wide, id_vars=["fertility_rate"], value_vars=years,
                            var_name='year', value_name='value')
df_fertility_long.head()

,fertility_rate,year,value
0,Afghanistan,1800,7.00
1,Albania,1800,4.60
2,Algeria,1800,6.99
3,Angola,1800,6.93
4,Antigua and Barbuda,1800,5.00


Vemos que estamos casi listos con el cambio. Solo es necesario cambiar los nombres de las columnas para poder usarlos más facilmente.

In [5]:
df_fertility_long = df_fertility_long.rename(columns={"fertility_rate":"country", "value":"fertility"})
df_fertility_long.head()

,country,year,fertility
0,Afghanistan,1800,7.00
1,Albania,1800,4.60
2,Algeria,1800,6.99
3,Angola,1800,6.93
4,Antigua and Barbuda,1800,5.00


Existen algunos valores que son `NaN`, y que conviene eliminar para evitar problemas en los cruces de datos. 

Finalmente, los años deberían ser números, así que tambien hacemos un casting de valores.

In [6]:
df_fertility = df_fertility_long.dropna()
df_fertility = df_fertility.astype({"country":str, "year":int, "fertility":float})
df_fertility

,country,year,fertility
0,Afghanistan,1800,7.00
1,Albania,1800,4.60
2,Algeria,1800,6.99
3,Angola,1800,6.93
4,Antigua and Barbuda,1800,5.00
5,Argentina,1800,6.80
6,Armenia,1800,7.80
7,Aruba,1800,5.64
8,Australia,1800,6.50
9,Austria,1800,5.10


Juntando los paises en un dataframe df.

In [8]:
df = df_gapminder.merge(df_fertility, how="left", on=["country","year"])
df.shape

(1704, 8)

Guardaremos los datos para usarlos en la parte 3, en formato csv y pickle, para tenerlos según sea conveniente.

In [9]:
df.to_pickle(os.path.join("data","extended_gapminder.pkl"))
df.to_csv(os.path.join("data","extended_gapminder.csv"), sep=";", index=False)

Veamos si los tamaños de los dataframes son consistentes:

In [10]:
print("Tamaño original DataFrame gapminder:", df_gapminder.shape)
print("Tamaño original DataFrame fertility:", df_fertility.shape)
print("Tamaño original DataFrame despues de merge:", df.shape)

Tamaño original DataFrame gapminder: (1704, 7)
Tamaño original DataFrame fertility: (43412, 3)
Tamaño original DataFrame despues de merge: (1704, 8)


¡Todo parece en orden!

## Actualizando el gráfico

### Pregunta 2.1 [15 puntos]
Actualice el gráfico interactivo de la parte 1 para tener:
- Eje $x$ con valores de fertilidad. 
- Eje $y$ con valores de esperanza de vida.
- La animación muestre los valores menor o igual al año seleccionado.

¡No olvides actualizar todos los textos que hayan cambiado!

In [4]:
# FIX ME #

## Pregunta 2.2 [15 puntos]
¿Cuales son los 3 países (del dataset gapminder) que experimentaron una mayor disminución en la fertilidad entre los años 1952 y 2007? ¿Existen países que hayan experimentado un aumento en la fertilidad?

Complete el código a continuación para obtener un dataframe y responda en el texto. Compruebe su respuesta directamente con los datos.

In [2]:
# FIX ME #

*R:* Los 3 países que experimentaron una mayor disminución en la fertilidad entre los años 1952 y 2007 son ...